In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 69.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 21.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, TFBertModel

In [4]:
# Load the dataset
train_df = pd.read_csv('/content/drive/MyDrive/NEEWWWWW/DATA_NEW/PC/1000/train_800.csv')
test_df = pd.read_csv('/content/drive/MyDrive/NEEWWWWW/DATA_NEW/PC/1000/test_200.csv')

In [5]:
from transformers import BertTokenizer, TFBertModel

# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

# print the configuration of the BERT model
print(bert_model.config)

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


BertConfig {
  "_name_or_path": "bert-base-cased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.28.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 28996
}



In [6]:
# Load the pre-trained BERT model and tokenizer
model_name = 'bert-base-cased'
tokenizer = BertTokenizer.from_pretrained(model_name)
bert_model = TFBertModel.from_pretrained(model_name)

# count total number of layers in the model
num_layers = bert_model.config.num_hidden_layers

# freeze first 6 hidden layers
for layer in bert_model.layers[0:6]:
    layer.trainable = False

# print which layers are frozen and which ones are not
for i, layer in enumerate(bert_model.layers):
    if i < 6:
        print(f"Layer {i}: Frozen")
    else:
        print(f"Layer {i}: Not Frozen")

# make sure the remaining layers after the 6th layer are trainable
for layer in bert_model.layers[6:]:
    layer.trainable = True

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Layer 0: Frozen


In [7]:
# Define the model
max_len = 16
input_ids = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
attention_mask = tf.keras.layers.Input(shape=(max_len,), dtype=tf.int32)
embedding = bert_model(input_ids, attention_mask)[1]
dense = tf.keras.layers.Dense(64, activation='relu')(embedding)
dropout = tf.keras.layers.Dropout(0.5)(dense)
output = tf.keras.layers.Dense(1, activation='sigmoid')(dropout)
model = tf.keras.models.Model(inputs=[input_ids, attention_mask], outputs=output)

In [8]:
# # Freeze the BERT model layers
# for layer in bert_model.layers:
#     layer.trainable = False

In [9]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(lr=1e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [10]:
# Tokenize the input data
train_inputs = tokenizer(list(train_df['text']), padding=True, truncation=True, max_length=max_len, return_tensors='tf')
test_inputs = tokenizer(list(test_df['text']), padding=True, truncation=True, max_length=max_len, return_tensors='tf')

In [11]:
# Convert the labels to arrays
train_labels = train_df['label'].values
test_labels = test_df['label'].values

In [ ]:
# Train the model
batch_size = 32
epochs = 200
model.fit([train_inputs['input_ids'], train_inputs['attention_mask']], train_labels, validation_data=([test_inputs['input_ids'], test_inputs['attention_mask']], test_labels), batch_size=batch_size, epochs=epochs)

Epoch 1/200
25/25 [==============================] - 80s 3s/step - loss: 0.7771 - accuracy: 0.5163 - val_loss: 0.6741 - val_accuracy: 0.5800
Epoch 2/200
25/25 [==============================] - 66s 3s/step - loss: 0.6816 - accuracy: 0.5713 - val_loss: 0.6718 - val_accuracy: 0.6100
Epoch 3/200
25/25 [==============================] - 56s 2s/step - loss: 0.6756 - accuracy: 0.5900 - val_loss: 0.6700 - val_accuracy: 0.6300
Epoch 4/200
25/25 [==============================] - 67s 3s/step - loss: 0.6738 - accuracy: 0.5888 - val_loss: 0.6764 - val_accuracy: 0.5350
Epoch 5/200
25/25 [==============================] - 56s 2s/step - loss: 0.6718 - accuracy: 0.5925 - val_loss: 0.6750 - val_accuracy: 0.5600
Epoch 6/200
25/25 [==============================] - 57s 2s/step - loss: 0.6611 - accuracy: 0.6187 - val_loss: 0.6601 - val_accuracy: 0.6200
Epoch 7/200
25/25 [==============================] - 56s 2s/step - loss: 0.6530 - accuracy: 0.6250 - val_loss: 0.6590 - val_accuracy: 0.5900
Epoch 8/200
2

In [ ]:
model.save('/content/gdrive/My Drive/keras_Bert_2.h5')